<a href="https://colab.research.google.com/github/hpmlinh26/DAILY-HABIT-TRACKER-THEO-D-I-GAMIFY-TH-I-QUEN-T-T/blob/main/Daily_Habit_Tracker_%E2%80%93_Theo_d%C3%B5i_%26_gamify_th%C3%B3i_quen_t%E1%BB%91t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Daily Habit Tracker – Theo dõi & gamify thói quen tốt


**Daily Habit Tracker & Gamify**
- Mục tiêu: Theo dõi thói quen, tính điểm, streak, leaderboard
- Công nghệ: Pandas + NumPy (core), Matplotlib (chart), Streamlit (web)


In [ ]:
# Cài thư viện cần thiết
!pip -q install streamlit pandas numpy matplotlib

In [ ]:
# Reset làm sạch dữ liệu
import shutil
from pathlib import Path
shutil.rmtree("data", ignore_errors=True)
Path("data").mkdir(exist_ok=True)
print("Reset xong data/")

Reset xong data/


In [38]:
# DAILY HABIT TRACKER — GAME MODE (Streamlit)
%%writefile app.py

# PHẦN 1 — IMPORT & LUẬT GAME
# Import thư viện + khai báo constant để đổi luật game/điểm nhanh.

import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
from pathlib import Path

COMPLETED = "Completed"
MISSED = "Missed"
FREQ = ["Daily", "Weekly"]

STREAK = {3: 10, 7: 30, 14: 50, 22: 100, 30: 150}
DAILY_CHALLENGE_ALL_FINISHED = 20
LEVEL_STEP = 100

DATA = "data"
Path(DATA).mkdir(exist_ok=True)

# PHẦN 2 — FIXED QUESTS + LOOT TABLE
# Quest cố định để app “game” đúng chất: ai vào cũng có bộ nhiệm vụ nền.

HABIT_TEMPLATES = [
    {"name": "Steps >= 8000",       "frequency": "Daily",  "points": 10},
    {"name": "Water >= 2000 ml",    "frequency": "Daily",  "points": 10},
    {"name": "Study >= 1 hour",     "frequency": "Daily",  "points": 10},
    {"name": "Sleep >= 7 hours",    "frequency": "Daily",  "points": 10},
    {"name": "Wake up <= 07:00",    "frequency": "Daily",  "points": 10},
]

LOOT_POOL = [
    {"item": "Coin",  "rarity": "Common",    "points": 10,  "weight": 0.70},
    {"item": "Coin",  "rarity": "Uncommon",  "points": 20,  "weight": 0.20},
    {"item": "Gem",   "rarity": "Rare",      "points": 50,  "weight": 0.08},
    {"item": "Badge", "rarity": "Epic",      "points": 100, "weight": 0.02},
]

BOSS_LOOT_POOL = [
    {"item": "Gem",   "rarity": "Rare",      "points": 80,  "weight": 0.55},
    {"item": "Badge", "rarity": "Epic",      "points": 150, "weight": 0.35},
    {"item": "Relic", "rarity": "Legendary", "points": 300, "weight": 0.10},
]


# PHẦN 3 — SCHEMA (DataFrames rỗng)
# Chuẩn hoá cột để load/save không vỡ.

def empty_users():
    return pd.DataFrame(columns=["user_id", "username", "created_at"])

def empty_habits():
    return pd.DataFrame(columns=[
        "habit_id", "user_id", "name", "frequency", "points",
        "active", "created_at", "is_fixed"
    ])

def empty_logs():
    return pd.DataFrame(columns=["log_id", "user_id", "habit_id", "log_date", "status", "note", "timestamp"])

def empty_rewards():
    # reward_type: daily_chest | boss_chest
    return pd.DataFrame(columns=["reward_id", "user_id", "reward_date", "reward_type", "item", "rarity", "points", "timestamp"])


# PHẦN 4 — I/O CSV (Load/Save)
# CSV nhẹ, dễ demo, dễ nhìn dữ liệu.

def now_iso():
    return pd.Timestamp.now().isoformat(timespec="seconds")

def p(filename: str) -> Path:
    return Path(DATA) / filename

def _drop_unnamed(df: pd.DataFrame) -> pd.DataFrame:
    # Mấy file CSV hay dính cột Unnamed: 0 (do save nhầm index), drop cho sạch.
    if df is None or df.empty:
        return df
    return df.loc[:, ~df.columns.astype(str).str.contains(r"^Unnamed")].copy()

def _load_one(file: str, empty_fn):
    fp = p(file)
    return pd.read_csv(fp) if fp.exists() else empty_fn()

def load_all():
    users   = _drop_unnamed(_load_one("users.csv", empty_users))
    habits  = _drop_unnamed(_load_one("habits.csv", empty_habits))
    logs    = _drop_unnamed(_load_one("logs.csv", empty_logs))
    rewards = _drop_unnamed(_load_one("rewards.csv", empty_rewards))

    # Vá cột thiếu (lỡ bạn chạy bản cũ chưa có is_fixed/active)
    if "active" not in habits.columns:
        habits["active"] = 1
    if "is_fixed" not in habits.columns:
        habits["is_fixed"] = 0

    # Ép kiểu tối thiểu để tránh lỗi
    if not users.empty:
        users["user_id"] = pd.to_numeric(users["user_id"], errors="coerce").fillna(0).astype(int)
        users["username"] = users["username"].fillna("").astype(str).str.strip()

    if not habits.empty:
        for c in ["habit_id", "user_id", "points", "active", "is_fixed"]:
            if c not in habits.columns:
                habits[c] = 0
            habits[c] = pd.to_numeric(habits[c], errors="coerce").fillna(0).astype(int)
        habits["name"] = habits.get("name", "").fillna("").astype(str).str.strip()
        habits["frequency"] = habits.get("frequency", "Daily").fillna("Daily").astype(str)

    if not logs.empty:
        for c in ["log_id", "user_id", "habit_id"]:
            if c not in logs.columns:
                logs[c] = 0
            logs[c] = pd.to_numeric(logs[c], errors="coerce").fillna(0).astype(int)
        logs["status"] = logs.get("status", MISSED).fillna(MISSED).astype(str)
        logs["note"] = logs.get("note", "").fillna("").astype(str)
        logs["log_date"] = pd.to_datetime(logs.get("log_date", pd.NaT), errors="coerce").dt.normalize()
        logs = logs.dropna(subset=["log_date"]).reset_index(drop=True)

    if not rewards.empty:
        for c in ["reward_id", "user_id", "points"]:
            if c not in rewards.columns:
                rewards[c] = 0
            rewards[c] = pd.to_numeric(rewards[c], errors="coerce").fillna(0).astype(int)
        rewards["reward_date"] = pd.to_datetime(rewards.get("reward_date", pd.NaT), errors="coerce").dt.normalize()
        rewards["reward_type"] = rewards.get("reward_type", "").fillna("").astype(str)
        rewards["item"] = rewards.get("item", "").fillna("").astype(str)
        rewards["rarity"] = rewards.get("rarity", "").fillna("").astype(str)

    return users, habits, logs, rewards

def save_all(users, habits, logs, rewards):
    users.to_csv(p("users.csv"), index=False)
    habits.to_csv(p("habits.csv"), index=False)

    out_logs = logs.copy()
    if not out_logs.empty:
        out_logs["log_date"] = pd.to_datetime(out_logs["log_date"]).dt.strftime("%Y-%m-%d")
    out_logs.to_csv(p("logs.csv"), index=False)

    out_rewards = rewards.copy()
    if not out_rewards.empty:
        out_rewards["reward_date"] = pd.to_datetime(out_rewards["reward_date"]).dt.strftime("%Y-%m-%d")
    out_rewards.to_csv(p("rewards.csv"), index=False)


# PHẦN 5 — DATA LAYER (CRUD)
# Gom thao tác vào hàm -> UI gọi gọn, sửa logic dễ.

def next_id(df, col):
    if df is None or df.empty or col not in df.columns:
        return 1
    return int(pd.to_numeric(df[col], errors="coerce").fillna(0).max()) + 1

def norm(x) -> str:
    return str(x).strip()

def create_user(users, username: str):
    # Chống trùng theo chuẩn: strip + lower
    username = norm(username)
    if not username:
        raise ValueError("Username rỗng")

    existed = users["username"].fillna("").astype(str).str.strip().str.lower()
    if (existed == username.lower()).any():
        i = 2
        sug = f"{username}_{i}"
        while (existed == sug.lower()).any():
            i += 1
            sug = f"{username}_{i}"
        raise ValueError(f"Username đã tồn tại. Gợi ý: {sug}")

    row = {"user_id": next_id(users, "user_id"), "username": username, "created_at": now_iso()}
    return pd.concat([users, pd.DataFrame([row])], ignore_index=True)

def ensure_fixed_habits(habits, user_id: int):
    # Đảm bảo mỗi user có 1 bộ quest nền, chỉ tạo thiếu chứ không tạo lại.
    uid = int(user_id)
    cur = habits[habits["user_id"] == uid] if not habits.empty else pd.DataFrame()
    existing = set(cur["name"].fillna("").astype(str).str.strip().str.lower().tolist()) if not cur.empty else set()

    hid = next_id(habits, "habit_id")
    rows = []
    for h in HABIT_TEMPLATES:
        nm = h["name"].strip().lower()
        if nm in existing:
            continue
        rows.append({
            "habit_id": hid,
            "user_id": uid,
            "name": h["name"],
            "frequency": h["frequency"],
            "points": int(h["points"]),
            "active": 1,
            "created_at": now_iso(),
            "is_fixed": 1
        })
        hid += 1

    if rows:
        habits = pd.concat([habits, pd.DataFrame(rows)], ignore_index=True)
        return habits, True
    return habits, False

def toggle_habit(habits, habit_id: int, active: int):
    habits = habits.copy()
    habits.loc[habits["habit_id"].astype(int) == int(habit_id), "active"] = int(active)
    return habits

def update_points(habits, habit_id: int, points: int):
    habits = habits.copy()
    habits.loc[habits["habit_id"].astype(int) == int(habit_id), "points"] = int(points)
    return habits

def upsert_log(logs, user_id: int, habit_id: int, log_date, status: str, note: str=""):
    # Upsert: cùng user-habit-ngày thì ghi đè (đỡ rác dữ liệu)
    uid = int(user_id); hid = int(habit_id)
    d = pd.to_datetime(log_date).normalize()

    if status not in [COMPLETED, MISSED]:
        raise ValueError("status phải là Completed/Missed")

    if not logs.empty:
        same = (
            (logs["user_id"] == uid) &
            (logs["habit_id"] == hid) &
            (pd.to_datetime(logs["log_date"]).dt.normalize() == d)
        )
        logs = logs.loc[~same].copy()

    row = {
        "log_id": next_id(logs, "log_id"),
        "user_id": uid,
        "habit_id": hid,
        "log_date": d,
        "status": status,
        "note": str(note),
        "timestamp": now_iso()
    }
    return pd.concat([logs, pd.DataFrame([row])], ignore_index=True)

def delete_log(logs, user_id: int, habit_id: int, log_date):
    if logs.empty:
        return logs
    uid = int(user_id); hid = int(habit_id)
    d = pd.to_datetime(log_date).normalize()
    mask = (
        (logs["user_id"] == uid) &
        (logs["habit_id"] == hid) &
        (pd.to_datetime(logs["log_date"]).dt.normalize() == d)
    )
    return logs.loc[~mask].copy()

def delete_all_logs_of_day(logs, user_id: int, log_date):
    if logs.empty:
        return logs
    uid = int(user_id)
    d = pd.to_datetime(log_date).normalize()
    mask = (logs["user_id"] == uid) & (pd.to_datetime(logs["log_date"]).dt.normalize() == d)
    return logs.loc[~mask].copy()


# PHẦN 6 — GAME LOGIC
# Chỉ tập trung tính toán: streak / progress / chest / boss / points.

def milestone_bonus(streak_len: int) -> int:
    total = 0
    for k in sorted(STREAK.keys()):
        if streak_len >= k:
            total += int(STREAK[k])
    return total

def current_streak_daily(logs, user_id: int, habit_id: int) -> int:
    df = logs[
        (logs["user_id"] == int(user_id)) &
        (logs["habit_id"] == int(habit_id)) &
        (logs["status"] == COMPLETED)
    ].copy()
    if df.empty:
        return 0

    df["log_date"] = pd.to_datetime(df["log_date"]).dt.normalize()
    dates = sorted(df["log_date"].unique())

    streak = 1
    for i in range(len(dates) - 1, 0, -1):
        if (pd.to_datetime(dates[i]) - pd.to_datetime(dates[i-1])).days == 1:
            streak += 1
        else:
            break
    return int(streak)

def daily_progress(habits, logs, user_id: int, day):
    day = pd.to_datetime(day).normalize()
    daily_active = habits[
        (habits["user_id"] == int(user_id)) &
        (habits["active"] == 1) &
        (habits["frequency"] == "Daily")
    ][["habit_id"]]
    need = int(len(daily_active))
    if need == 0:
        return 0, 0

    done = logs[
        (logs["user_id"] == int(user_id)) &
        (pd.to_datetime(logs["log_date"]).dt.normalize() == day) &
        (logs["status"] == COMPLETED)
    ][["habit_id"]].merge(daily_active, on="habit_id", how="inner")

    return need, int(done["habit_id"].nunique())

def has_claimed(rewards, user_id: int, day, reward_type: str):
    if rewards.empty:
        return False
    day = pd.to_datetime(day).normalize()
    return not rewards[
        (rewards["user_id"] == int(user_id)) &
        (pd.to_datetime(rewards["reward_date"]).dt.normalize() == day) &
        (rewards["reward_type"] == reward_type)
    ].empty

def roll_loot(pool):
    w = np.array([x["weight"] for x in pool], dtype=float)
    w = w / w.sum()
    idx = int(np.random.choice(np.arange(len(pool)), p=w))
    return pool[idx].copy()

def open_daily_chest(rewards, user_id: int, day):
    day = pd.to_datetime(day).normalize()
    loot = roll_loot(LOOT_POOL)
    total_points = int(DAILY_CHALLENGE_ALL_FINISHED + loot["points"])
    row = {
        "reward_id": next_id(rewards, "reward_id"),
        "user_id": int(user_id),
        "reward_date": day,
        "reward_type": "daily_chest",
        "item": loot["item"],
        "rarity": loot["rarity"],
        "points": total_points,
        "timestamp": now_iso()
    }
    return pd.concat([rewards, pd.DataFrame([row])], ignore_index=True), row

def week_start(d):
    d = pd.to_datetime(d).normalize()
    return d - pd.Timedelta(days=int(d.weekday()))

def boss_progress(habits, logs, user_id: int, day):
    day = pd.to_datetime(day).normalize()
    ws = week_start(day)
    we = ws + pd.Timedelta(days=6)

    daily_active = habits[
        (habits["user_id"] == int(user_id)) &
        (habits["active"] == 1) &
        (habits["frequency"] == "Daily")
    ][["habit_id"]]
    daily_cnt = int(len(daily_active))
    if daily_cnt == 0:
        return ws, we, 0, 0, 0

    done = logs[
        (logs["user_id"] == int(user_id)) &
        (pd.to_datetime(logs["log_date"]).dt.normalize() >= ws) &
        (pd.to_datetime(logs["log_date"]).dt.normalize() <= we) &
        (logs["status"] == COMPLETED)
    ][["habit_id"]].merge(daily_active, on="habit_id", how="inner")

    damage = int(len(done))
    target = daily_cnt * 5
    return ws, we, daily_cnt, damage, target

def has_claimed_boss(rewards, user_id: int, day):
    if rewards.empty:
        return False
    ws = week_start(day)
    return not rewards[
        (rewards["user_id"] == int(user_id)) &
        (pd.to_datetime(rewards["reward_date"]).dt.normalize() == ws) &
        (rewards["reward_type"] == "boss_chest")
    ].empty

def open_boss_chest(rewards, user_id: int, day):
    ws = week_start(day)
    loot = roll_loot(BOSS_LOOT_POOL)
    row = {
        "reward_id": next_id(rewards, "reward_id"),
        "user_id": int(user_id),
        "reward_date": ws,
        "reward_type": "boss_chest",
        "item": loot["item"],
        "rarity": loot["rarity"],
        "points": int(loot["points"]),
        "timestamp": now_iso()
    }
    return pd.concat([rewards, pd.DataFrame([row])], ignore_index=True), row

def points_summary(habits, logs, rewards, user_id: int):
    uid = int(user_id)

    # Base points: cộng điểm của quest mỗi lần Completed
    base = 0
    if not logs.empty and not habits.empty:
        done = logs[(logs["user_id"] == uid) & (logs["status"] == COMPLETED)].merge(
            habits[habits["active"] == 1][["habit_id", "points"]],
            on="habit_id", how="left"
        )
        done["points"] = done["points"].fillna(0).astype(int)
        base = int(done["points"].sum())

    # Streak bonus: dựa trên streak hiện tại của từng daily quest
    streak_pts = 0
    my_daily = habits[(habits["user_id"] == uid) & (habits["active"] == 1) & (habits["frequency"] == "Daily")]
    if not my_daily.empty and not logs.empty:
        for hid in my_daily["habit_id"].astype(int).tolist():
            streak_pts += milestone_bonus(current_streak_daily(logs, uid, hid))

    # Reward points: daily_chest + boss_chest
    reward_pts = 0
    if not rewards.empty:
        reward_pts = int(rewards[(rewards["user_id"] == uid) &
                                 (rewards["reward_type"].isin(["daily_chest", "boss_chest"]))]["points"].sum())

    total = base + streak_pts + reward_pts
    level = int(total // LEVEL_STEP) if LEVEL_STEP > 0 else 0
    return base, streak_pts, reward_pts, total, level

def points_by_day(user_id: int, habits, logs, rewards, days=30):
    uid = int(user_id)
    end = pd.Timestamp.today().normalize()
    start = end - pd.Timedelta(days=days - 1)
    idx = pd.date_range(start, end, freq="D")
    s = pd.Series(0, index=idx)

    # Logs points/day
    if not logs.empty and not habits.empty:
        df = logs[(logs["user_id"] == uid) & (logs["status"] == COMPLETED)].copy()
        df["log_date"] = pd.to_datetime(df["log_date"]).dt.normalize()
        df = df[(df["log_date"] >= start) & (df["log_date"] <= end)]
        if not df.empty:
            m = df.merge(habits[habits["active"] == 1][["habit_id", "points"]], on="habit_id", how="left")
            m["points"] = m["points"].fillna(0).astype(int)
            s = s.add(m.groupby("log_date")["points"].sum().reindex(idx, fill_value=0), fill_value=0)

    # Reward points/day
    if not rewards.empty:
        r = rewards[(rewards["user_id"] == uid) & (rewards["reward_type"].isin(["daily_chest", "boss_chest"]))].copy()
        r["reward_date"] = pd.to_datetime(r["reward_date"]).dt.normalize()
        r = r[(r["reward_date"] >= start) & (r["reward_date"] <= end)]
        if not r.empty:
            s = s.add(r.groupby("reward_date")["points"].sum().reindex(idx, fill_value=0), fill_value=0)

    return s

def last7_marks(user_id: int, habit_id: int, logs):
    today = pd.Timestamp.today().normalize()
    days = [today - pd.Timedelta(days=i) for i in range(6, -1, -1)]
    df = logs[(logs["user_id"] == int(user_id)) & (logs["habit_id"] == int(habit_id))].copy()
    if df.empty:
        return ["⬜"] * 7
    df["log_date"] = pd.to_datetime(df["log_date"]).dt.normalize()
    mp = df.sort_values("timestamp").groupby("log_date")["status"].last().to_dict()
    return ["✅" if (d in mp and mp[d] == COMPLETED) else "⬜" for d in days]

def daily_done_count(user_id: int, logs, habits):
    if logs.empty or habits.empty:
        return pd.Series(dtype=int)

    daily_ids = habits[
        (habits["user_id"] == int(user_id)) &
        (habits["active"] == 1) &
        (habits["frequency"] == "Daily")
    ][["habit_id"]]
    if daily_ids.empty:
        return pd.Series(dtype=int)

    df = logs[(logs["user_id"] == int(user_id)) & (logs["status"] == COMPLETED)].copy()
    if df.empty:
        return pd.Series(dtype=int)

    df["log_date"] = pd.to_datetime(df["log_date"]).dt.normalize()
    df = df.merge(daily_ids, on="habit_id", how="inner")
    return df.groupby("log_date")["habit_id"].nunique().astype(int).sort_index()

def build_calendar_30d(user_id: int, logs: pd.DataFrame, habits: pd.DataFrame, days: int = 30) -> pd.DataFrame:
    """
    Lịch 30 ngày dạng bảng (Mon..Sun).
    """
    end = pd.Timestamp.today().normalize()
    start = end - pd.Timedelta(days=days - 1)

    # số daily quests đang bật (để tính % hoàn thành trong ngày)
    daily_active = habits[
        (habits["user_id"] == int(user_id)) &
        (habits["active"] == 1) &
        (habits["frequency"] == "Daily")
    ]
    need = int(len(daily_active))

    # series: mỗi ngày completed bao nhiêu daily quest
    s = daily_done_count(user_id, logs, habits)  # bạn đã có hàm này
    s.index = pd.to_datetime(s.index).normalize()

    # căn lịch theo tuần (Mon..Sun)
    start2 = start - pd.Timedelta(days=int(start.weekday()))
    end2 = end + pd.Timedelta(days=int(6 - end.weekday()))
    full_idx = pd.date_range(start2, end2, freq="D")

    def icon(v: int) -> str:
        if need == 0:
            return "⬜"
        r = v / need
        if v == 0:
            return "⬜"      # 0%
        if r < 0.34:
            return "🟥"      # thấp
        if r < 0.67:
            return "🟧"      # trung bình
        if r < 1.0:
            return "🟨"      # gần đủ
        return "🟩"          # 100%

    cells = []
    for d in full_idx:
        if d < start or d > end:
            cells.append("")  # phần đệm ngoài 30 ngày
        else:
            v = int(s.get(d, 0))
            cells.append(f"{d.day:02d} {icon(v)}")

    mat = np.array(cells).reshape(-1, 7)
    cal = pd.DataFrame(mat, columns=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"])
    return cal



# PHẦN 7 — UI (Streamlit)
# Lưu ý: set_page_config phải chạy trước mọi st.* để tránh warning/bug.

st.set_page_config(page_title="Habit Game", layout="wide")
st.title("Daily Habit Tracker — Game Mode 🎮")

users_df, habits_df, logs_df, rewards_df = load_all()

# ===== Sidebar: Account =====
st.sidebar.header("Account")

with st.sidebar.expander("➕ Tạo user mới", expanded=False):
    with st.form("signup"):
        new_name = st.text_input("Username")
        ok = st.form_submit_button("Create")
    if ok:
        try:
            users_df = create_user(users_df, new_name)

            # Lấy user_id vừa tạo (đỡ lệch do lower/strip)
            uid_new = int(users_df["user_id"].max())

            habits_df, added = ensure_fixed_habits(habits_df, uid_new)
            save_all(users_df, habits_df, logs_df, rewards_df)

            st.success("Đã tạo user + gán quest ✅")
            st.rerun()
        except Exception as e:
            st.error(str(e))

names = users_df["username"].tolist()
selected = st.sidebar.selectbox("Chọn user", names) if names else None
if not selected:
    st.info("Chưa có user. Hãy tạo user ở sidebar.")
    st.stop()

user_id = int(users_df.loc[users_df["username"] == selected, "user_id"].iloc[0])

# Đảm bảo user luôn có fixed quests (chỉ save khi có thêm)
habits_df, added = ensure_fixed_habits(habits_df, user_id)
if added:
    save_all(users_df, habits_df, logs_df, rewards_df)

show_lb = st.sidebar.checkbox("Hiện Leaderboard (toàn hệ thống)", value=False)
menu = ["🏠 Home", "✅ Today", "🛠️ Manage", "📈 Charts"]
if show_lb:
    menu.append("🏆 Leaderboard")
page = st.sidebar.radio("Menu", menu)

# ===== Header nhỏ: tình trạng hôm nay (đặt đúng chỗ, có đủ biến) =====
need_today, done_today = daily_progress(habits_df, logs_df, user_id, pd.Timestamp.today().date())
st.caption(f"Hôm nay: {done_today}/{need_today} daily quests đã hoàn thành.")

# PAGE: HOME
if page == "🏠 Home":
    base, streak_pts, reward_pts, total, level = points_summary(habits_df, logs_df, rewards_df, user_id)

    st.subheader(f"Xin chào, **{selected}** 👋")
    c1, c2, c3, c4, c5 = st.columns(5)
    c1.metric("Total", total)
    c2.metric("Base", base)
    c3.metric("Streak Bonus", streak_pts)
    c4.metric("Rewards", reward_pts)
    c5.metric("Level", level)

    if LEVEL_STEP > 0:
        cur = total % LEVEL_STEP
        st.progress(cur / LEVEL_STEP)
        st.caption(f"Còn **{LEVEL_STEP - cur}** điểm để lên level tiếp theo.")

    # Weekly Boss
    st.markdown("### 🐉 Weekly Boss")
    today = pd.Timestamp.today().normalize()
    ws, we, daily_cnt, dmg, target = boss_progress(habits_df, logs_df, user_id, today)

    if daily_cnt == 0:
        st.info("Bạn đang tắt hết Daily quests, boss ngủ 😴")
    else:
        st.write(f"Week: **{ws.date()} → {we.date()}**")
        st.progress(min(1.0, dmg / max(1, target)))
        st.write(f"Damage: **{dmg}/{target}**")

        claimed = has_claimed_boss(rewards_df, user_id, today)
        if dmg >= target and not claimed:
            st.success("Boss defeated! Mở Boss Chest 🎁")
            if st.button("Open Boss Chest"):
                rewards_df, rr = open_boss_chest(rewards_df, user_id, today)
                save_all(users_df, habits_df, logs_df, rewards_df)
                st.balloons()
                st.success(f"Bạn nhận: **{rr['item']}** ({rr['rarity']}) — +{rr['points']} pts")
                st.rerun()
        elif claimed:
            st.info("Boss Chest tuần này đã mở ✅")
        else:
            st.warning("Chưa đủ damage. Cứ hoàn thành daily để tích damage!")

    # Inventory
    st.markdown("### 🎒 Inventory")
    inv = rewards_df[rewards_df["user_id"] == user_id].copy()
    if inv.empty:
        st.info("Chưa có loot.")
    else:
        inv_sum = inv.groupby(["reward_type", "item", "rarity"]).size().reset_index(name="qty")
        st.dataframe(inv_sum.sort_values(["reward_type", "rarity", "item"]), use_container_width=True)

    # Quests 7 ngày gần nhất
    st.markdown("### 📌 Quests (7 ngày gần nhất)")
    my = habits_df[(habits_df["user_id"] == user_id) & (habits_df["active"] == 1)].copy().sort_values("habit_id")
    if my.empty:
        st.warning("Bạn đang tắt hết quests.")
    else:
        for r in my.itertuples(index=False):
            marks = last7_marks(user_id, r.habit_id, logs_df)
            st.write(f"**{r.name}** • {int(r.points)} pts • {' '.join(marks)}")

# PAGE: TODAY
elif page == "✅ Today":
    st.subheader("Today — Daily Quest List ✅")

    chosen = st.date_input("Chọn ngày", value=pd.Timestamp.today().date())
    day = pd.to_datetime(chosen).normalize()

    daily = habits_df[
        (habits_df["user_id"] == user_id) &
        (habits_df["active"] == 1) &
        (habits_df["frequency"] == "Daily")
    ].copy().sort_values("habit_id")

    if daily.empty:
        st.info("Bạn đang tắt hết Daily quests.")
        st.stop()

    need, done = daily_progress(habits_df, logs_df, user_id, day)
    st.progress(done / max(1, need))
    st.write(f"Progress: **{done}/{need}**")

    day_logs = logs_df[
        (logs_df["user_id"] == user_id) &
        (pd.to_datetime(logs_df["log_date"]).dt.normalize() == day)
    ].copy()

    with st.form("today_checkin"):
        status_map, note_map = {}, {}

        for r in daily.itertuples(index=False):
            hid = int(r.habit_id)
            default = day_logs[day_logs["habit_id"] == hid]
            default_status = default["status"].iloc[0] if not default.empty else MISSED
            default_note = default["note"].iloc[0] if not default.empty else ""

            c1, c2, c3 = st.columns([2.6, 1, 2.4])
            with c1:
                st.write(f"**{r.name}**  ·  +{int(r.points)} pts")
            with c2:
                ok_done = st.checkbox("Done", value=(default_status == COMPLETED), key=f"done_{hid}_{day}")
            with c3:
                note = st.text_input("Note", value=str(default_note), key=f"note_{hid}_{day}")

            status_map[hid] = COMPLETED if ok_done else MISSED
            note_map[hid] = note

        ok = st.form_submit_button("Save")

    if ok:
        for hid, status in status_map.items():
            logs_df = upsert_log(logs_df, user_id, hid, chosen, status, note_map.get(hid, ""))
        save_all(users_df, habits_df, logs_df, rewards_df)
        st.success("Saved ✅")
        st.rerun()

    # Daily Chest
    st.markdown("---")
    st.markdown("### 🎁 Daily Chest")
    need2, done2 = daily_progress(habits_df, logs_df, user_id, day)
    claimed = has_claimed(rewards_df, user_id, day, "daily_chest")

    if done2 == need2 and not claimed:
        st.success("Hoàn thành đủ Daily quests! Mở rương 🎁")
        if st.button("Open Daily Chest"):
            rewards_df, rr = open_daily_chest(rewards_df, user_id, day)
            save_all(users_df, habits_df, logs_df, rewards_df)
            st.balloons()
            st.success(f"Bạn nhận: **{rr['item']}** ({rr['rarity']}) — +{rr['points']} pts")
            st.rerun()
    elif claimed:
        st.info("Hôm nay đã mở rương ✅")
    else:
        st.warning("Chưa đủ điều kiện (phải hoàn thành tất cả Daily quests).")

# PAGE: MANAGE
elif page == "🛠️ Manage":
    st.subheader("Manage — Quests & Logs")
    tab_quests, tab_logs = st.tabs(["🧩 Quests", "🗓️ Logs"])

    with tab_quests:
        st.markdown("### Quản lý Quests (bật/tắt & chỉnh điểm)")
        my = habits_df[habits_df["user_id"] == user_id].copy().sort_values(["frequency", "habit_id"])
        st.dataframe(my[["habit_id", "name", "frequency", "points", "active"]], use_container_width=True)

        pick = st.selectbox(
            "Chọn quest",
            my["habit_id"].astype(int).tolist(),
            format_func=lambda hid: f"#{hid} — {my.loc[my['habit_id']==hid,'name'].iloc[0]}",
            key="pick_q"
        )
        cur = my[my["habit_id"] == int(pick)].iloc[0]

        c1, c2 = st.columns(2)
        with c1:
            active = st.checkbox("Active", value=int(cur["active"]) == 1, key="q_active")
        with c2:
            pts = st.number_input("Points", min_value=0, value=int(cur["points"]), step=1, key="q_pts")

        if st.button("Save Quest", key="save_q"):
            habits_df = toggle_habit(habits_df, int(pick), 1 if active else 0)
            habits_df = update_points(habits_df, int(pick), int(pts))
            save_all(users_df, habits_df, logs_df, rewards_df)
            st.success("Updated ✅")
            st.rerun()

    with tab_logs:
        st.markdown("### Quản lý Logs (xem / sửa (upsert) / xóa)")
        st.caption("Sửa log = tick lại Done/Missed rồi Save (hệ thống sẽ upsert).")

        chosen = st.date_input("Chọn ngày", value=pd.Timestamp.today().date(), key="m_day")
        day = pd.to_datetime(chosen).normalize()

        day_logs = logs_df[
            (logs_df["user_id"] == user_id) &
            (pd.to_datetime(logs_df["log_date"]).dt.normalize() == day)
        ].copy()

        if not day_logs.empty:
            day_logs = day_logs.merge(habits_df[["habit_id", "name", "points"]], on="habit_id", how="left")
            st.dataframe(day_logs[["habit_id", "name", "status", "note", "timestamp"]], use_container_width=True)
        else:
            st.info("Ngày này chưa có log (bạn vẫn có thể tạo nhanh bên dưới).")

        my_active = habits_df[(habits_df["user_id"] == user_id) & (habits_df["active"] == 1)].copy().sort_values("habit_id")
        if my_active.empty:
            st.warning("Bạn đang tắt hết quests.")
        else:
            pick_hid = st.selectbox(
                "Chọn quest để sửa/xóa log",
                my_active["habit_id"].astype(int).tolist(),
                format_func=lambda hid: f"#{hid} — {my_active.loc[my_active['habit_id']==hid,'name'].iloc[0]}",
                key="pick_log"
            )

            cur_row = day_logs[day_logs["habit_id"] == int(pick_hid)] if not day_logs.empty else pd.DataFrame()
            default_status = cur_row["status"].iloc[0] if not cur_row.empty else MISSED
            default_note = cur_row["note"].iloc[0] if not cur_row.empty else ""

            c1, c2 = st.columns([1, 3])
            with c1:
                done = st.checkbox("Done", value=(default_status == COMPLETED), key="edit_done")
            with c2:
                note = st.text_input("Note", value=str(default_note), key="edit_note")

            a, b, c = st.columns(3)
            with a:
                if st.button("Save Log (Upsert)", key="save_log"):
                    logs_df = upsert_log(logs_df, user_id, int(pick_hid), chosen, COMPLETED if done else MISSED, note)
                    save_all(users_df, habits_df, logs_df, rewards_df)
                    st.success("Saved ✅")
                    st.rerun()
            with b:
                if st.button("Delete Log", key="del_log"):
                    logs_df = delete_log(logs_df, user_id, int(pick_hid), chosen)
                    save_all(users_df, habits_df, logs_df, rewards_df)
                    st.warning("Deleted ✅")
                    st.rerun()
            with c:
                if st.button("Delete ALL logs of this day", key="del_all_day"):
                    logs_df = delete_all_logs_of_day(logs_df, user_id, chosen)
                    save_all(users_df, habits_df, logs_df, rewards_df)
                    st.warning("Deleted all logs of the day ✅")
                    st.rerun()

# PAGE: CHARTS
elif page == "📈 Charts":
    st.subheader("Points per day (30 ngày gần nhất) — gồm cả Rewards")

    s = points_by_day(user_id, habits_df, logs_df, rewards_df, days=30)
    fig = plt.figure()
    plt.plot(s.index, s.values)
    plt.xticks(rotation=45)
    plt.title("Points per day (30 days)")
    plt.tight_layout()
    st.pyplot(fig)

    # Heatmap để đúng chỗ (không render lung tung ở mọi page)
    st.markdown("### 📅 Lịch 30 ngày")
    cal = build_calendar_30d(user_id, logs_df, habits_df, days=30)
    st.table(cal)
    st.caption("Legend: ⬜ 0% | 🟥 <34% | 🟧 34–66% | 🟨 67–99% | 🟩 100% daily quests trong ngày")



# PAGE: LEADERBOARD
else:
    st.subheader("Leaderboard (toàn hệ thống)")

    rows = []
    for u in users_df.itertuples(index=False):
        base, streak_pts, reward_pts, total, level = points_summary(habits_df, logs_df, rewards_df, int(u.user_id))
        rows.append({
            "user_id": int(u.user_id),
            "username": str(u.username),
            "base": base,
            "streak": streak_pts,
            "rewards": reward_pts,
            "total": total,
            "level": level
        })

    lb = pd.DataFrame(rows).sort_values(["total", "username"], ascending=[False, True]).reset_index(drop=True)
    st.dataframe(lb, use_container_width=True)


Overwriting app.py


In [ ]:
!pip -q install streamlit pandas numpy matplotlib
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 \
  --server.enableCORS false --server.enableXsrfProtection false > streamlit.log 2>&1 &

!nohup ./cloudflared tunnel --url http://localhost:8501 --no-autoupdate > cloudflared.log 2>&1 &


cloudflared: Text file busy


In [37]:
import re
from pathlib import Path
from IPython.display import display, Markdown

log = Path("cloudflared.log").read_text(errors="ignore")
m = re.search(r"https://[-0-9a-z]*\.trycloudflare\.com", log)

if m:
    url = m.group(0)
    display(Markdown(f"### Mở web tại đây: [{url}]({url})"))
else:
    print("Chưa thấy link trong cloudflared.log. Hãy chạy lại grep/tail hoặc đợi vài giây.")


### Mở web tại đây: [https://forty-providers-marco-blocked.trycloudflare.com](https://forty-providers-marco-blocked.trycloudflare.com)